In [1]:
using Luna
using  DelimitedFiles 
import PyPlot:pygui, plt
close("all")
pygui(true)

pres= 3.0
gas = :Ar

in_dir="input"
L = 3e-3
p_ed = 1e-3

file_ρ    = "dens_$(pres)bar.dat"  # name of density profile data file 
path_ρ    = joinpath(in_dir, file_ρ)                  # sys. path to density profile data file 

z_in = readdlm(path_ρ,' ', Float64, '\n')[:,1]        # read in spatial points 
ρ_in = readdlm(path_ρ,' ', Float64, '\n')[:,2]        # read in density data 
z_coms0 = maximum(z_in)/2 - 1.5e-3

dens_coms = Maths.CSpline(z_in, ρ_in)                      # interpolate density function 
    
Z= (0, L/2, L)                                        # define points for pressure gradient
P= (p_ed, pres, p_ed)  # define values for pressure gradient (see "p_const" above)
(coren,dens_grad)=Capillary.gradient(gas,Z,P)              # gives n(ω; z) and ρ(z) from pressure profile   
       
zarr = range(0, L, 100)

ρ_grad =   [dens_grad(z) for z in zarr]
ρ_coms=    [dens_coms(z_coms0+z) for z in zarr]

plt.title("Comparison of COMSOL and gradient profiles for $(gas) at $(pres)bar")
plt.plot(zarr*1e3, ρ_grad, label="gradient", color="grey")
plt.plot(zarr*1e3, ρ_coms, label="COMSOL", color="red")
plt.xlabel("Cell position (mm)")
plt.ylabel("Number density (1/m³)")
plt.legend()
plt.savefig(joinpath("output", "COMS_grad_comp.png"), dpi=1000)
plt.show()


In [33]:
gas_arr = [:Ar, :Ne, :He, :Kr, :Xe, :N2, :N2O]
dens_grad_arr = [Capillary.gradient(gas,Z,P)[2] for gas in gas_arr]
c_arr = ["black", "grey", "blue", "cyan", "orange", "green", "purple"]

plt.title("Comparison of different gas profiles at $(pres)bar")
plt.xlabel("Cell position (mm)")
plt.ylabel("Number density (1/m³)")
for i in 1:length(gas_arr)
    ρ_grad = [dens_grad_arr[i](z) for z in zarr]
    plt.plot(zarr*1e3, ρ_grad, label="$(gas_arr[i])", color=c_arr[i])
end   
plt.plot(zarr*1e3, ρ_coms, label="COMSOL", color="red")
plt.legend()
plt.savefig(joinpath("output", "gas_dens_comp.png"), dpi=1000)
plt.show()
